<a href="https://colab.research.google.com/github/ArasyH/Agentic-Ai-Workshop-Challenge/blob/main/agentic_patterns_challenge_arasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ARASY HEMASTHIAR
## AGENTIC AI WORKSHOP CHALLENGE

In [2]:
# !pip install openai-agents
!pip install requests
!pip install groq
!pip install langchain
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.6/129.6 kB 3.3 MB/s eta 0:00:00


In [6]:
import os
import json
import asyncio
import requests

from pydantic import BaseModel
from typing import List, Any
# from agents import Agent, Runner, function_tool, trace
from langchain_groq import ChatGroq

from google.colab import userdata

SECTORS_API_KEY = userdata.get('SECTORS_API_KEY')
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

headers = {"Authorization": SECTORS_API_KEY}

In [7]:
def retrieve_from_endpoint(url: str) -> dict:

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.HTTPError as err:
        raise SystemExit(err)
    return json.dumps(data)

In [12]:
from langchain_core.tools import tool


@tool
def get_company_overview(ticker: str, country: str) -> str | None:
    """
    Get company overview from Singapore Exchange (SGX) or Indonesia Exchange (IDX) in specific industries.
    """
    assert country.lower() in ["indonesia", "singapore", "malaysia"], "Country must be either Indonesia, Singapore, or Malaysia"

    if(country.lower() == "indonesia"):
        url = f"https://api.sectors.app/v1/company/report/{ticker}/?sections=overview"
    if(country.lower() == "singapore"):
        url = f"https://api.sectors.app/v1/sgx/company/report/{ticker}/"
    if(country.lower() == "malaysia"):
        url = f"https://api.sectors.app/v1/klse/company/report/{ticker}/"

    try:
        return retrieve_from_endpoint(url)
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

@tool
def get_top_companies_ranked(dimension: str) -> List[str]:
    """
    Return a detail list of top companies (symbol) based on certain dimension (dividend yield, total dividend, revenue, earnings, market_cap, PB ratio, PE ratio, or PS ratio).
    """

    url = f"https://api.sectors.app/v1/companies/top/?classifications={dimension}"

    return retrieve_from_endpoint(url)

# tool
# def csv_export_tool(data: object) -> str:
#   """
#   Convert the object to csv format
#   """
#   import pandas as pd
#   # Convert object to dataframe
#   df = pd.DataFrame.from_dict(data, orient='index')

#   #Convert dataframe to csv saved as 'export.csv'
#   df.to_csv('export.csv', index=True)
#   return "Successfully exported to export.csv"


In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_groq import ChatGroq
from langchain.agents import create_tool_calling_agent, AgentExecutor

llm = ChatGroq(model_name="llama3-8b-8192", temperature=0, groq_api_key = GROQ_API_KEY)

# class ValidTickers(BaseModel):
#     tickers: List[str]


#Agen 1 Screening perusahaan terbaik berdasarkan dimensi tertentu
screener_tools = [get_top_companies_ranked]
screener_prompt = ChatPromptTemplate.from_messages([
    ("system", """Get the top companies based on the given metric. Return the tickers of the top companies, without the .JK suffix. Return in a list."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

get_top_companies_based_on_metric_agent = create_tool_calling_agent(llm, screener_tools, screener_prompt)
get_top_companies_based_on_metric_executor = AgentExecutor(agent=get_top_companies_based_on_metric_agent, tools=screener_tools, verbose=True, return_intermediate_steps=True)

# #Agen 2 determiner agent
# determiner_tools = []
# determiner_prompt = ChatPromptTemplate.from_messages([
#     ("system", """Generate a list of tickers (symbols) of companies to research based on the query.
#     Tickers on IDX are exactly 4 characters long, e.g. BBCA, BBRI, TLKM"""),
#     ("placeholder", "{agent_scratchpad}")

# ])
# determine_companies_to_research_agent = create_tool_calling_agent(llm, determiner_tools, determiner_prompt)
# determine_companies_to_research_executor = AgentExecutor(agent=determine_companies_to_research_agent, tools=determiner_tools, verbose=True)


# Agen 3 Researcher agent
research_tools = [get_company_overview]
researcher_prompt = ChatPromptTemplate.from_messages([
    ("system", """Research the company using the right tool."""),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

company_research_agent = create_tool_calling_agent(llm, research_tools, researcher_prompt)
company_research_agent_executor = AgentExecutor(agent=company_research_agent, tools=research_tools, verbose=True, return_intermediate_steps=True)


In [14]:
 """
 CHALLENGE:
  Modify the code here so that you're not hardcoding one stock ticker at a time
  (who have time for that?!) Instead, use a Multi-Agent workflow to have another
  Agent generate a list of companies that you want to research, based on some
  screening condition, then delegate to the second Agent to do the work.
"""

async def main():
    input_prompt = input("🤖: What kind of companies are you interested in? \n💀: ")

    print("\n--- LANGKAH 1: Screening Perusahaan (Agen 1) ---")
    response_agent_1 = await get_top_companies_based_on_metric_executor.ainvoke({"input": input_prompt})

    # --- PERBAIKAN UTAMA DI SINI ---
    try:
        # 1. Ambil langkah perantara dari Agen 1
        intermediate_step = response_agent_1['intermediate_steps'][0]
        tool_output_json_string = intermediate_step[1]

        # 2. Parse string JSON mentah dari tool
        api_data = json.loads(tool_output_json_string)

        # 3. Ekstrak data ticker dari struktur JSON yang kompleks
        #    Kunci 'market_cap' mungkin berubah sesuai dimensi, jadi kita ambil kunci pertama dari dictionary
        dimension_key = list(api_data.keys())[0]
        companies_list = api_data[dimension_key]

        # 4. Ambil simbolnya saja dan hapus .JK
        top_companies_tickers = [company['symbol'].replace('.JK', '') for company in companies_list]

    except (KeyError, IndexError, json.JSONDecodeError) as e:
        print(f"Gagal mem-parsing output dari Agen 1: {e}")
        print(f"Raw output: {response_agent_1}")
        return
    # ---------------------------------

    print(f"\n✅: Agen 1 menemukan tickers: {top_companies_tickers}")

    # --- LANGKAH 2 (Kode ini sudah benar dari perbaikan sebelumnya) ---
    print("\n--- LANGKAH 2: Riset Detail per Perusahaan (Agen 2) ---")
    all_results = {}
    for ticker in top_companies_tickers:
        print(f"\n🔎: Getting information on: {ticker}")
        research_input = f"Get an overview for the company with ticker '{ticker}' in Indonesia."

        company_research_result = await company_research_agent_executor.ainvoke({"input": research_input})

        try:
            intermediate_step_2 = company_research_result['intermediate_steps'][0]
            tool_output_json_string_2 = intermediate_step_2[1]
            detailed_info = json.loads(tool_output_json_string_2)
            all_results[ticker] = detailed_info

            print(f"🤖: Info untuk {ticker} (Data dari API):")
            print(json.dumps(detailed_info, indent=2))
        except (KeyError, IndexError, json.JSONDecodeError) as e:
            print(f"Gagal mengambil atau mem-parsing detail untuk {ticker}. Error: {e}")
            print(f"Raw agent output: {company_research_result}")

    print(f"\n✅: Selesai! Informasi untuk '{input_prompt}' telah disajikan.")

await main()

🤖: What kind of companies are you interested in? 
💀: Top Companies with best Dividend Yield

--- LANGKAH 1: Screening Perusahaan (Agen 1) ---


> Entering new AgentExecutor chain...

Invoking: `get_top_companies_ranked` with `{'dimension': 'dividend_yield'}`


{"dividend_yield": [{"symbol": "DMAS.JK", "company_name": "PT Puradelta Lestari Tbk.", "dividend_yield": 0.207142857142857}, {"symbol": "BJBR.JK", "company_name": "Bank Pembangunan Daerah Jawa Barat dan Banten Tbk", "dividend_yield": 0.100887573964497}, {"symbol": "BNGA.JK", "company_name": "PT Bank CIMB Niaga Tbk", "dividend_yield": 0.0902782583927763}, {"symbol": "NISP.JK", "company_name": "PT Bank OCBC NISP Tbk", "dividend_yield": 0.0776556776556777}, {"symbol": "ASII.JK", "company_name": "Astra International Tbk", "dividend_yield": 0.0669565217391304}]}
Invoking: `get_top_companies_ranked` with `{'dimension': 'dividend_yield'}`


{"dividend_yield": [{"symbol": "DMAS.JK", "company_name": "PT Puradelta Lestari Tbk.", "dividend_

Success Response Api about Classification:
- Market Cap
- Dividend Yield


List that Couldnt provide information about classification:
- P/B Ratio
- Earnings
- Revenue
- P/E
- Ps
-